In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from experiments.Field_Switching import FieldSwitchingExperiment, extract_data
from pycontrol.filters.io import WriteToHDF5
from pycontrol.filters.plot import Plotter
import numpy as np
import matplotlib.pyplot as plt

In [3]:
exp = FieldSwitchingExperiment()
exp.sample = "CSHE-5 C2R3"
exp.comment = "Field Sweep"
exp.measure_current.value = 3e-6
fields = np.linspace(-0.005,0.03,200)
fields = np.append(fields, np.flipud(fields))
main_sweep = exp.add_sweep(exp.field,fields)
exp.init_instruments()

In [4]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C2R3\CSHE5-C2R3-FieldSweep_2016-07-26.h5")
fig = Plotter(name="CSHE5-C2R3 - Field Sweep", plot_dims=1)
edges = [(exp.resistance, wr.data), (exp.resistance, fig.data)]
exp.set_graph(edges)

In [5]:
exp.init_progressbar(num=1,notebook=True)
exp.run_sweeps()
exp.shutdown_instruments()

No more data for plotter
Killing bokeh server thread 7492
Killing child process 5240


In [6]:
field, res = extract_data(wr.filename)

In [7]:
fig = plt.figure()
plt.plot(field, res/1e3,'-o')
plt.xlabel('Field (T)', size=16)
plt.ylabel('Resistance (kOhm)', size=16)
plt.title('Field Sweep - CSHE5 - C2R3', size=16)
plt.show()

In [8]:
# Estimate TMR
TMR = (np.max(res)-np.min(res))/np.min(res)
print("TMR = {:.0f}%".format(100*TMR))

TMR = 59%


In [9]:
# Find the center of the loop of coercivity
mid = np.mean([np.max(res),np.min(res)])
transition_pos = field[np.where(np.diff(np.sign(res - mid)))[0][0]]
transition_neg = field[np.where(np.diff(np.sign(res - mid)))[0][-1]]
print(transition_pos)
print(transition_neg)
print("Center is at {:.1f} G".format(1e4*(transition_neg + transition_pos)*0.5))
print("Coercivity is {:.1f} G".format(1e4*(transition_pos - transition_neg)*0.5))

0.0171608040201
-0.000954773869347
Center is at 81.0 G
Coercivity is 90.6 G
